In [1]:
from unyt import *
from unyt.dimensions import *
import numpy as np
from numpy import zeros, cos, sin
from numpy.linalg import inv

In [8]:
def out_of_bounds(val, mn, mx, condition):
    """Check if value satisfies boundary conditions.

    Parameters
    ----------
    val : float or int
        value to evaluate against boundary conditions
    mn, mx: float or int
        the minimum and maximum boundaries
    condition: {'g', 'ge', 'g-l', 'ge-l', 'g-le', 'ge-le', 'l', 'le'}
        the boundary condition to evaluate

    Returns
    -------
    bool
        True if ``val`` is outside boundaries, False if within boundaries

    Notes
    -----
    ``condition`` should may be any of the values defined for each of the
    boundary conditions described in the table below:

    =========== ==================
    Value       Boundary Condition
    =========== ==================
    ``'g'``     ``val > mn``
    ``'ge'``    ``val >= mn``
    ``'g-l'``   ``mn < val < mx``
    ``'ge-l'``  ``mn <= val < mx``
    ``'g-le'``  ``mn < val <= mx``
    ``'ge-le'`` ``mn <= val <= mx``
    ``'l'``     ``val < mx``
    ``'le'``    ``val <= mx``
    =========== ==================

    """

    return not {
        'g': lambda: val > mn and mx is None,
        'ge': lambda: val >= mn and mx is None,
        'g-l': lambda: mn < val < mx,
        'ge-l': lambda: mn <= val < mx,
        'g-le': lambda: mn < val <= mx,
        'ge-le': lambda: mn <= val <= mx,
        'l': lambda: mn is None and val < mx,
        'le': lambda: mn is None and val <= mx
    }.get(condition)()


In [2]:
UREG = unit_registry.default_unit_registry
UREG.add(
    symbol='strain',
    base_value=1.0,
    dimensions=length / length,
    tex_repr=r"\rm{\varepsilon}",
    prefixable=True
)
# add percent for coefficient of hygroscopic expansion
UREG.add(
    symbol='percent',
    base_value=1.0,
    dimensions=dimensionless,
    tex_repr=r'\rm{%}',
    prefixable=False
)
# add torque units
UREG.add(
    symbol='N_m',
    base_value=1.0,
    dimensions=force * length,
    tex_repr=r'\rm{N \cdot m}',
    prefixable=True
)
UREG.add(
    symbol='in_lbf',
    base_value=(1 * inch * lbf).to('N*m').v.item(0),
    dimensions=force * length,
    tex_repr=r'\rm{in \cdot lb_{F}}',
    prefixable=True
)
UREG.add(
    symbol='ft_lbf',
    base_value=(1 * ft * lbf).to('N*m').v.item(0),
    dimensions=force * length,
    tex_repr=r'\rm{ft \cdot lb_{f}}'
)

us_unit_system = UnitSystem(
    name='us',
    length_unit='inch',
    mass_unit='lb',
    time_unit='s',
    temperature_unit='degF',
    angle_unit='degree',
    registry=UREG
)
us_unit_system['force'] = lbf
us_unit_system['pressure'] = psi

si_unit_system = UnitSystem(
    name='si',
    length_unit='m',
    mass_unit='kg',
    time_unit='s',
    temperature_unit='degC',
    angle_unit='rad',
    registry=UREG
)

USYS = us_unit_system

In [176]:
class UnitDimensionError(ValueError):
    """Error raised if wrong unit type is assigned.

    Parameters
    ----------
    name : str
        name of entity to print in error message
    correct_dim : unyt.dimensions.dimension
        the unit that should have been assigned

    """

    def __init__(self, name, correct_dim):
        self.name = name
        self.correct_dim = correct_dim

    def __str__(self):
        return (
            f"`{self.name}` must have units with dimensions {self.correct_dim}"
        )

class BoundedValueError(ValueError):
    """Error for values that are out of specified boundaries.

    Parameters
    ----------
    name : string
        name of object checked for boundary
    mn : float
        the minimum value checked against
    mx : float
        the maximum value checked against
    condition: {'g', 'ge', 'g-l', 'ge-l', 'g-le', 'ge-le', 'l', 'le'}
        the condition ``name`` was evaluated against ;see 'Notes' for
        acceptable values

    Notes
    -----
    ``condition`` should may be any of the values defined for each of the
    boundary conditions described in the table below:

    =========== ==================
    Value       Boundary Condition
    =========== ==================
    ``'g'``     ``val > mn``
    ``'ge'``    ``val >= mn``
    ``'g-l'``   ``mn < val < mx``
    ``'ge-l'``  ``mn <= val < mx``
    ``'g-le'``  ``mn < val <= mx``
    ``'ge-le'`` ``mn <= val <= mx``
    ``'l'``     ``val < mx``
    ``'le'``    ``val <= mx``
    =========== ==================

    """

    def __init__(self, name, mn, mx, condition):
        self.name = name
        self.both = True

        if mn is not None:
            self.mn = str(mn) + ' '
        else:
            self.mn = ''
            self.both = False

        if mx is not None:
            self.mx = str(mx) + ' '
        else:
            self.mx = ''
            self.both = False

        args = {
            'g': {
                'mn': str(mn) + ' ',
                'mn_eq': '>',
                'mx': '',
                'mx_eq': '',
                'nd': ''
            },
            'ge': {
                'mn': str(mn) + ' ',
                'mn_eq': '>=',
                'mx': '',
                'mx_eq': '',
                'nd': ''
            },
            'g-l': {
                'mn': str(mn) + ' ',
                'mn_eq': '>',
                'mx': str(mx) + ' ',
                'mx_eq': '<',
                'nd': 'and '
            },
            'ge-l': {
                'mn': str(mn) + ' ',
                'mn_eq': '>=',
                'mx': str(mx) + ' ',
                'mx_eq': '<',
                'nd': 'and '
            },
            'g-le': {
                'mn': str(mn) + ' ',
                'mn_eq': '>',
                'mx': str(mx) + ' ',
                'mx_eq': '<=',
                'nd': 'and '
            },
            'ge-le': {
                'mn': str(mn) + ' ',
                'mn_eq': '>=',
                'mx': str(mx) + ' ',
                'mx_eq': '<= ',
                'nd': 'and '
            },
            'l': {
                'mn': '',
                'mn_eq': '',
                'mx': str(mx) + ' ',
                'mx_eq': '<',
                'nd': ''
            },
            'le': {
                'mn': '',
                'mn_eq': '',
                'mx': str(mx) + ' ',
                'mx_eq': '<=',
                'nd': ''
            }
        }.get(condition)

        for each in args:
            self.__setattr__(each, args[each])


    def __str__(self):
        return (
            f"`{self.name}` must be a number {self.mn_eq}{self.mn}{self.nd}"
            + f"{self.mx_eq}{self.mx}"
        )

In [177]:
class Lamina:
    
    _param_dims = {
        't': (length,),
        'E1': (pressure,),
        'E2': (pressure,),
        'nu12': (dimensionless,),
        'G12': (pressure,),
        'a11': (1 / temperature,),
        'a22': (1 / temperature,),
        'b11': (dimensionless,),  # percent moisture change
        'b22': (dimensionless,),
        'F1': (dimensionless, pressure),  # dimensionless for strain
        'F2': (dimensionless, pressure),
        'F12': (dimensionless, pressure)
    }
    # define attribute limits for use with `out_of_bounds()`
    _param_limits = {
        't': {'mn': 0, 'mx': None, 'condition': 'g'},
        'E1': {'mn': 0, 'mx': None, 'condition': 'g'},
        'E2': {'mn': 0, 'mx': None, 'condition': 'g'},
        'nu12': {'mn': 0, 'mx': 1, 'condition': 'g-l'},
        'G12': {'mn': 0, 'mx': None, 'condition': 'g'}
    }
    __slots__ = tuple(_param_dims) + ('usys',)

    def __init__(self, t, E1, E2, nu12, G12, a11, a22, b11, b22, F1=0, F2=0,
                 F12=0, usys=USYS):

        self.t = t
        self.E1 = E1
        self.E2 = E2
        self.nu12 = nu12
        self.G12 = G12
        self.a11 = a11
        self.a22 = a22
        self.b11 = b11
        self.b22 = b22
        self.F1 = F1
        self.F2 = F2
        self.F12 = F12
        super().__setattr__('usys', usys)

    def __setattr__(self, name, attr):
        """Extend __setattr__() to validate units."""
        # set dimensions for required attributes
        if name in self._param_dims:
            correct_dim = self._param_dims.get(name)
            # check if object has units
            try:
                attr.units

            # if no units assigned, assign units
            except AttributeError:
                attr *= USYS[correct_dim[0]]  # assign first

            # if units assigned, check units have correct dimensionality
            else:
                for each in correct_dim:
                    if attr.units.dimensions != each:
                        raise UnitDimensionError(name, correct_dim)
                    else:
                        # units are converted to master unit system for
                        # consistency when creating unit_arrays in `Ply` class
                        attr.convert_to_base(self.usys)

        # make sure value is within limits
        if name in self._param_limits:
            if out_of_bounds(attr.value, **self._param_limits.get(name)):
                raise BoundedValueError(name, **self._param_limits.get(name))

        # catch unit system change and convert all attributes with units
        if name == 'usys':
            for each in self._param_dims:
                getattr(self, each).convert_to_base(attr)

        super().__setattr__(name, attr)
        
    def __repr__(self):
        r = f"{type(self).__name__}:"
        for each in self.__slots__:
            r += f"\n    {each + ':':<6}{getattr(self, each)}"
        return r

In [182]:
x = Lamina(1, 2, 3 * si_unit_system['force'], 0.4, 5, 6, 7, 8, 9)
print(x)
x.usys = si_unit_system
print(x)


UnitDimensionError: `E2` must have units with dimensions ((mass)/((length)*(time)**2),)

In [172]:
class Laminate:
    def __init__(self, dT, dM):
        self.dT = dT
        self.dM = dM

class Ply(Lamina):
    _param_dims = {
        **Lamina._param_dims,
        'theta': (angle,),
        'z': (length,),
        'e_m': (dimensionless,),
        's_m': (pressure,),
        'Q': (pressure,),
        'Qbar': (pressure,),
        'T': (dimensionless,),
        'e_t':(dimensionless,),
        'e_h':(dimensionless,),
        's_t':(pressure,),
        's_h':(pressure,),
    }

    _base_attr = tuple(_param_dims)
    _calc_attr = ('Q', 'Qbar', 'T', 'Tinv', 'e_t', 'e_h', 's_t', 's_h',
                  'laminate', 'failure_theory', 'failure_index', 'usys')
                  # `usys` is intended to only be assigned by the Laminate
    __slots__ = _base_attr + _calc_attr + ('__locked',)

    def __unlock(locked_func):
        def unlocked_func(self, *args, **kwargs):
            super().__setattr__('_Ply__locked', False)
            locked_func(self, *args, **kwargs)
            super().__setattr__('_Ply__locked', True)
        return unlocked_func

    @__unlock
    def __init__(self, laminate, t, theta, E1, E2, nu12, G12, a11, a22, b11,
                 b22, F1=0, F2=0, F12=0, usys=USYS, failure_theory='strain'):
        """Extend ``__init__`` to account for Ply-only attributes."""

        self.laminate = laminate
        self.z = 0
        self.theta = theta
        self.e_m = zeros((3, 1))
        self.e_t = zeros((3, 1))
        self.e_h = zeros((3, 1))
        self.s_m = zeros((3, 1))
        self.s_t = zeros((3, 1))
        self.s_h = zeros((3, 1))
        self.failure_theory = failure_theory
        self.failure_index = 0

        super().__init__(t, E1, E2, nu12, G12, a11, a22, b22, b22, F1, F2, F12,
                         usys)
        self.__update()

    def __setattr__(self, attr, val):
        """Extend ``__setattr__`` to protect calculated attributes."""

        if self.__locked:
            # udpate ply and laminate after updated properties are set
            if attr in self._base_attr:
                super().__setattr__(attr, val)
                self.__update()

            # don't set protected values
            elif attr in self._calc_attr:
                raise AttributeError(type(self).__name__ + ".%s" % attr
                                     + " is a derived value and cannot be set")

        else:
            super().__setattr__(attr, val)
        
    @__unlock
    def __update(self):
        """Update calculated attributes."""

        # on-axis reduced stiffness matrix, Q
        # NASA-RP-1351, Eq (15)
        nu21 = self.nu12 * self.E2 / self.E1  # Jones, Eq (2.67)
        q11 = self.E1 / (1 - self.nu12 * nu21)
        q12 = self.nu12 * self.E2 / (1 - self.nu12 * nu21)
        q22 = self.E2 / (1 - self.nu12 * nu21)
        q66 = self.G12

        self.Q = unyt_array([[q11, q12, 0], [q12, q22, 0], [0, 0, q66]],
                            self.usys['pressure'])

        # the transformation matrix and its inverse
        # create intermediate trig terms
        m = cos(self.theta)
        n = sin(self.theta)

        # create transformation matrix and inverse
        self.T = unyt_array([[m**2, n**2, 2 * m * n],
                             [n**2, m**2, -2 * m * n],
                             [-m * n, m * n, m**2 - n**2]])
        self.Tinv = inv(self.T)

        # the transformed reduced stiffness matrix (laminate coordinate system)
        # Jones, Eq (2.84)
        self.Qbar = (self.Tinv @ self.Q @ self.Tinv.T) * self.usys['pressure']

        # thermal and hygroscopic strains in laminate and lamina c-systems
        # NASA-RP-1351 Eq (90), (91), and (95)
        self.e_t = (unyt_array([[self.a11], [self.a22], [0]])
                    / self.usys['temperature']) * self.laminate.dT
        self.e_h = (unyt_array([[self.b11], [self.b22], [0]])
                    / self.usys['dimensionless']) * self.laminate.dM

        # thermal and hygroscopic stresses in laminate and lamina c-systems
        # NASA-RP-1351 Eq (90), (91), and (95)
        self.s_t = (self.Q @ self.e_t)
        self.s_h = (self.Q @ self.e_h)

        # calculate failure index
        

    @classmethod
    def from_lamina(cls, lamina, laminate, theta):
        return cls(laminate=laminate,
                   theta=theta,
                   t=lamina.t,
                   E1=lamina.E1,
                   E2=lamina.E2,
                   nu12=lamina.nu12,
                   G12=lamina.G12,
                   a11=lamina.a11,
                   a22=lamina.a22,
                   b11=lamina.b11,
                   b22=lamina.b22,
                   F1=lamina.F1,
                   F2=lamina.F2,
                   F12=lamina.F12,
                   usys=lamina.usys)

    @property
    def zk(self):
        """The vertical location of the lamina's top plane."""
        return self.z + self.t / 2

    @property
    def zk1(self):
        """The vertical location of the lamina's bottom plane."""
        return self.z - self.t / 2

    @zk1.setter
    def zk1(self, new_zk1):
        self.z = new_zk1 + self.t / 2

    @property
    def e(self):
        """Total strain."""
        return self.e_m + self.e_t + self.e_h

    @property
    def s(self):
        """Total stress."""
        return self.s_m + self.s_t + self.s_h
    
    def __repr__(self):
        r = f'{type(self).__name__}'
        for each in sorted(self._param_dims):
            attr = getattr(self, each)
            if issubclass(attr.__class__, np.ndarray):
                lines = attr.__str__().splitlines()
                r += f"\n    {each + ':':<7}{lines[0]}"
                for i in range(1, len(lines)):
                    r += "\n{0:<11}{1:}".format(' ', lines[i])

            else:
                r += f"\n    {each + ':':<7}{attr}"
        r += f"\n    {'failure index' + ': '}{getattr(self, 'failure_index')}"
        r += f"\n    {'failure theory' + ': '}{getattr(self, 'failure_theory')}"
        return r

In [222]:
Q = unyt_array(np.ones((3, 3)), psi)
e = unyt_array(np.ones((3, 1)), dimensionless)
t = 1 * inch
N = (Q @ e) * (2 * t**2 - t**2)
print(N)

[[3.]
 [3.]
 [3.]] inch**2*psi


In [195]:
lam = Laminate((100 * R), 100)
pl = Ply(lam, 1, 45, 2, 3, 0.4, 5, 6e-6, 6e-6, 7e-6, 7e-6)
print(pl)
pl.usys = si_unit_system
print(pl)


Ply
    E1:    2 psi
    E2:    3 psi
    F1:    0 dimensionless
    F12:   0 dimensionless
    F2:    0 dimensionless
    G12:   5 psi
    Q:     [[2.63157895 1.57894737 0.        ]
            [1.57894737 3.94736842 0.        ]
            [0.         0.         5.        ]] psi
    Qbar:  [[ 7.43421141 -2.56578978 -0.3289474 ]
            [-2.56578978  7.43421141 -0.3289474 ]
            [-0.3289474  -0.3289474   0.85526322]] psi
    T:     [[ 0.49999997  0.49999997  0.99999997]
            [ 0.49999997  0.49999997 -0.99999997]
            [-0.49999997  0.49999997  0.        ]] dimensionless
    a11:   6e-06 1/degF
    a22:   6e-06 1/degF
    b11:   7e-06 dimensionless
    b22:   7e-06 dimensionless
    e_h:   [[0.0007]
            [0.0007]
            [0.    ]] dimensionless
    e_m:   [[0.]
            [0.]
            [0.]] dimensionless
    e_t:   [[0.0006]
            [0.0006]
            [0.    ]] dimensionless
    nu12:  0.4 dimensionless
    s_h:   [[0.00294737]
            

AttributeError: Ply.usys is a derived value and cannot be set